In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, count, when
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.builder \
    .appName("Create DataFrames from Logs") \
    .getOrCreate()

file_paths_http = ["./data/task1/00/http.log.gz", "./data/task1/01/http.log.gz", "./data/task1/02/http.log.gz",
                   "./data/task1/03/http.log.gz", "./data/task1/04/http.log.gz", "./data/task1/05/http.log.gz"]

file_paths_dns = ["./data/task1/00/dns.log.gz", "./data/task1/01/dns.log.gz", "./data/task1/02/dns.log.gz",
                  "./data/task1/03/dns.log.gz", "./data/task1/04/dns.log.gz", "./data/task1/05/dns.log.gz"]

df_http = spark.read.json(file_paths_http).withColumn("status_code", col("status_code").cast("int"))

df_dns = spark.read.json(file_paths_dns)

df_http.printSchema()
df_dns.printSchema()

df_http.show(5, truncate=True)
df_dns.show(5, truncate=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/06 06:42:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/06 06:42:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


root
 |-- host: string (nullable = true)
 |-- id.orig_h: string (nullable = true)
 |-- id.orig_p: long (nullable = true)
 |-- id.resp_h: string (nullable = true)
 |-- id.resp_p: long (nullable = true)
 |-- method: string (nullable = true)
 |-- orig_filenames: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- orig_fuids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- orig_mime_types: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- origin: string (nullable = true)
 |-- proxied: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- referrer: string (nullable = true)
 |-- request_body_len: long (nullable = true)
 |-- resp_filenames: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- resp_fuids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- resp_mime_types: array (nullable = true)
 |    |-- element: string (containsNull = true)

24/04/06 06:42:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------+---------------+---------+---------------+---------+------+--------------+--------------------+---------------+------+-------+--------------------+----------------+--------------+--------------------+---------------+-----------------+-----------+-----------------+----+-----------+---------------+------------------+--------------------+--------------------+--------+-------+
|           host|      id.orig_h|id.orig_p|      id.resp_h|id.resp_p|method|orig_filenames|          orig_fuids|orig_mime_types|origin|proxied|            referrer|request_body_len|resp_filenames|          resp_fuids|resp_mime_types|response_body_len|status_code|       status_msg|tags|trans_depth|             ts|               uid|                 uri|          user_agent|username|version|
+---------------+---------------+---------+---------------+---------+------+--------------+--------------------+---------------+------+-------+--------------------+----------------+--------------+------------------

In [3]:
df_http = df_http.withColumn("ts", to_timestamp(df_http["ts"]))
df_dns = df_dns.withColumn("ts", to_timestamp(df_dns["ts"]))

df_http.show(5)
df_dns.show(5)

+---------------+---------------+---------+---------------+---------+------+--------------+--------------------+---------------+------+-------+--------------------+----------------+--------------+--------------------+---------------+-----------------+-----------+-----------------+----+-----------+--------------------+------------------+--------------------+--------------------+--------+-------+
|           host|      id.orig_h|id.orig_p|      id.resp_h|id.resp_p|method|orig_filenames|          orig_fuids|orig_mime_types|origin|proxied|            referrer|request_body_len|resp_filenames|          resp_fuids|resp_mime_types|response_body_len|status_code|       status_msg|tags|trans_depth|                  ts|               uid|                 uri|          user_agent|username|version|
+---------------+---------------+---------+---------------+---------+------+--------------+--------------------+---------------+------+-------+--------------------+----------------+--------------+--------

In [4]:
df_http.createOrReplaceTempView("http_log")
df_dns.createOrReplaceTempView("dns_log")


In [11]:
http_log_filtered = spark.sql("""
    select 
        uri, 
        count(*) as accessed_count
    from 
        http_log
    where 
        status_code = 200 and method = 'GET'
    group by 
        uri
    order by 
        accessed_count desc
""")

http_log_filtered.show(5)
print(http_log_filtered.count())

+--------------------+--------------+
|                 uri|accessed_count|
+--------------------+--------------+
|                   /|          9475|
|/admin/config.php...|           556|
|  /main.php?logout=1|           194|
|/top.php?stuff=15...|           191|
|            /top.php|           179|
+--------------------+--------------+
only showing top 5 rows



7369


In [6]:
spark.sql("""
select 
    h.uid,
    d.uid,
    d.proto
from
    http_log h, 
    dns_log d
where
    trim(h.uid) = trim(d.uid)
""").show()


+------------------+------------------+-----+
|               uid|               uid|proto|
+------------------+------------------+-----+
| C1odva1GOl62Mubql| C1odva1GOl62Mubql|  tcp|
| CQLw22eZ8JIDpJcik| CQLw22eZ8JIDpJcik|  udp|
|CKaLaf1peiUU3kmTV8|CKaLaf1peiUU3kmTV8|  udp|
|CtEYyv1YQSocDA5oY6|CtEYyv1YQSocDA5oY6|  udp|
|CwrlVC14uolzZt7dPc|CwrlVC14uolzZt7dPc|  udp|
| CzIyt32dXmMFfK0cI| CzIyt32dXmMFfK0cI|  udp|
|Cvhzsd2JH09vFeDbJk|Cvhzsd2JH09vFeDbJk|  udp|
|CPUpzb4oqzPYQZxnF7|CPUpzb4oqzPYQZxnF7|  udp|
|CFWPlA4sGS8yUlrM7e|CFWPlA4sGS8yUlrM7e|  udp|
| CFDlRY1TJ52tSimA6| CFDlRY1TJ52tSimA6|  udp|
|CxWRZL3Ho8bdyWEiwk|CxWRZL3Ho8bdyWEiwk|  udp|
|C0cFv53bfGO02GQsma|C0cFv53bfGO02GQsma|  tcp|
| CscT2UI8EmU58fDuk| CscT2UI8EmU58fDuk|  tcp|
| CUYrwDbRuXBxapQFa| CUYrwDbRuXBxapQFa|  udp|
|CeiomQ3qdeS3ESqe99|CeiomQ3qdeS3ESqe99|  udp|
|ClveGS2nXhopfdBvoa|ClveGS2nXhopfdBvoa|  tcp|
| CxdESvjdAKHMAAvFg| CxdESvjdAKHMAAvFg|  udp|
| CqNa6ry5SSxMwkhYc| CqNa6ry5SSxMwkhYc|  udp|
|CoEJEF1ByMpkdNrZEd|CoEJEF1ByMpkdN

In [12]:
merged_df = spark.sql("""
    select 
        *
    from
        http_log h, 
        dns_log d
    where
        h.uid = d.uid
""")
merged_df = merged_df.drop(col("d.uid"))
merged_df.show(5)

+---------------+---------------+---------+---------------+---------+------+--------------+--------------------+---------------+------+-------+--------------------+----------------+--------------+--------------------+---------------+-----------------+-----------+-----------------+----+-----------+--------------------+------------------+--------------------+--------------------+--------+-------+-----+-----+----+-----+----+---+-------+--------------+---------+-------------+---------+-----+------+-----------+-----+----------+-------------------+-----+----------+--------+----+--------+--------------------+
|           host|      id.orig_h|id.orig_p|      id.resp_h|id.resp_p|method|orig_filenames|          orig_fuids|orig_mime_types|origin|proxied|            referrer|request_body_len|resp_filenames|          resp_fuids|resp_mime_types|response_body_len|status_code|       status_msg|tags|trans_depth|                  ts|               uid|                 uri|          user_agent|username|v

In [8]:
merged_df.select(["proto"]).dropDuplicates().show()

+-----+
|proto|
+-----+
|  tcp|
|  udp|
+-----+



In [29]:
uri_set = set(http_log_filtered.select("uri").rdd.flatMap(lambda x: x).collect())
filtered_df = merged_df.filter(col("uri").isin(uri_set))
grouped_df = filtered_df.groupBy("uri", "proto").agg(count("*").alias("count"))
tcp_df = grouped_df.filter(grouped_df["proto"] == "tcp")

total_counts = filtered_df.groupBy("uri").agg(count("*").alias("total_count"))
joined_df = tcp_df.join(total_counts, "uri")
result_df = joined_df.withColumn("tcp_percentage", (col("count") / col("total_count")) * 100)
result_df = result_df.select("uri", "tcp_percentage")


In [30]:
result_df.show(5, truncate = False)
result_df.count()

24/04/08 09:48:12 WARN DAGScheduler: Broadcasting large task binary with size 1385.3 KiB
24/04/08 09:48:12 WARN DAGScheduler: Broadcasting large task binary with size 1382.7 KiB
24/04/08 09:48:21 WARN DAGScheduler: Broadcasting large task binary with size 1385.0 KiB
24/04/08 09:48:21 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB


+-----------------------------------------+------------------+
|uri                                      |tcp_percentage    |
+-----------------------------------------+------------------+
|/lib/exe/indexer.php?id=start&1331903710 |100.0             |
|/lib/tpl/default/images/button-donate.gif|16.666666666666664|
|/lib/tpl/default/images/buttonshadow.png |50.0              |
|/lib/tpl/default/images/button-dw.png    |33.33333333333333 |
|/lib/tpl/default/images/button-xhtml.png |16.666666666666664|
+-----------------------------------------+------------------+
only showing top 5 rows



24/04/08 09:48:22 WARN DAGScheduler: Broadcasting large task binary with size 1383.0 KiB
24/04/08 09:48:22 WARN DAGScheduler: Broadcasting large task binary with size 1380.1 KiB
24/04/08 09:48:31 WARN DAGScheduler: Broadcasting large task binary with size 1387.1 KiB
24/04/08 09:48:31 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB


67

In [44]:
result_df_sql = spark.sql("""
    select
        h.uri,
        100.0 * sum(case when proto = "tcp" then 1 else 0 end) / count(*) as tcp_percentage
    from
        (
            select
                uri
            from
                http_log
            where 
                status_code = 200 
            and 
                method = 'GET'
            group by
                uri
        ) as selected_uris,
        http_log h,
        dns_log d
    where 
        h.uid = d.uid
    and 
        h.uri = selected_uris.uri
    group by
        h.uri
    order by
        tcp_percentage desc
""")
result_df_sql.show(68, truncate = False)
result_df_sql.count()

+--------------------------------------------------------------------------------------------+------------------+
|uri                                                                                         |tcp_percentage    |
+--------------------------------------------------------------------------------------------+------------------+
|/lib/exe/indexer.php?id=start&1331903710                                                    |100.00000000000000|
|/smconf.nsf                                                                                 |100.00000000000000|
|/classes/                                                                                   |100.00000000000000|
|/cgilib/                                                                                    |100.00000000000000|
|/_private/                                                                                  |100.00000000000000|
|/ccards/                                                                               

389

In [17]:
method_counts = df_http.groupBy("method").agg(count("*").alias("count"))

total_count = method_counts.selectExpr("sum(count) as total").collect()[0]["total"]

method_percentages = method_counts.withColumn("percentage", (col("count") / total_count) * 100)

method_percentages.show()

pie_data = method_percentages.toPandas()

+-----------+------+--------------------+
|     method| count|          percentage|
+-----------+------+--------------------+
|       some|    17|0.009147358565694177|
|     RDWWEM|     1|5.380799156290692E-4|
|    OPTIONS|  2388|  1.2849348385222175|
|     NESSUS|    13|0.006995038903177901|
|       POST| 40184|  21.622203329638516|
|       NULL|  1376|  0.7403979639055993|
|     DELETE|    22|0.011837758143839522|
|     Secure|    17|0.009147358565694177|
|   GNUTELLA|    34|0.018294717131388354|
|     MCPWYV|     1|5.380799156290692E-4|
|        PUT|    23|0.012375838059468592|
|   PROPFIND|    40| 0.02152319662516277|
|      TRACK|    13|0.006995038903177901|
|       HEAD|  4805|   2.585473994597678|
|      TRACE|    23|0.012375838059468592|
|        GET|136678|   73.54368670834992|
|     SEARCH|    72|0.038741753925292985|
|     EAILCX|     1|5.380799156290692E-4|
|   DESCRIBE|     4|0.002152319662516277|
|RPC_CONNECT|    35|0.018832797047017423|
+-----------+------+--------------

In [22]:
for method in pie_data["method"]:
    method_df = df_http.filter(df_http["method"] == method)
    status_counts = method_df.groupBy("status_code").agg(count("*").alias("count"))
    status_data = status_counts.toPandas()
    status_data["status_code"] = status_data["status_code"].fillna(0).astype(int)
    plt.figure(figsize=(8, 6), dpi = 160)
    plt.pie(status_data["count"], labels=status_data["status_code"])
    plt.title(f"Status Code Distribution for Method: {method}")
    plt.savefig(f"./output/task1/pie_chart_{method}.png")
    plt.close()
    print(f"Pie chart of method {method} generated.")

Pie chart of method some generated.


Pie chart of method RDWWEM generated.


Pie chart of method OPTIONS generated.


Pie chart of method NESSUS generated.


Pie chart of method POST generated.
Pie chart of method None generated.


...

Pie chart of method FPANCV generated.


Pie chart of method JZUIPY generated.


Pie chart of method GATMQW generated.


Pie chart of method EQAONR generated.


Pie chart of method WOIGAG generated.
